In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import math
from itertools import combinations, product

In this notebook, we will use the Multicommodity Flow formulation to get out optimal tours.

In [4]:
# Function will convert our datfile of x,y coordinates into dataframe: vertices
def TSP_vertices(file_name):
    data  = open(file_name, 'r')
    lines = data.readlines()[1:]
    x = []
    y = []
    for line in lines:
        p = line.split()
        x.append(float(p[0]))
        y.append(float(p[1]))
    data.close()
    # Transform into dataframe
    vertices = pd.DataFrame({'x':x, 'y':y})
    
    return vertices

In [5]:
# Will convert Euclidean distance for our coordinates as cost components for model
def euclideanDistMatrix(TSP):
    euclidean_cost = []
    for i in range(len(TSP)):
        row_distances = []
        for j in range(len(TSP)):
            row_distances.append(math.sqrt( (TSP['x'][i]-TSP['x'][j])**2 + (TSP['y'][i]-TSP['y'][j])**2 ))
        euclidean_cost.append(row_distances)
    return euclidean_cost 

In [6]:
def multiflow(file_name):

    # Turn .dat file into a cost matrix with all costs associated from traversing from i to j
    verticesDF = TSP_vertices(file_name)
    cost = euclideanDistMatrix(verticesDF)

    # Create a list V that contains all the vertices, numbered 0 through n-1
    V = []
    for i in range(len(verticesDF)):
        V.append(i)
    
    # Create a list A of all tuples of (i,j) which represent the arc from i to j
    # Create a list Y that represents flow of commodity k through edge (i,j) 
    A = [(i, j) for i in V for j in V if i != j]
    Y = [(i, j ,k) for i in V for j in V for k in V[1:] if i != j]
    
    # Instantiate the Model
    model = Model('Multi Flow')
    model.ModelSense = GRB.MINIMIZE

    # Instantiate the decision variables
    x = model.addVars(A, vtype = GRB.BINARY, name='edge')
    y = model.addVars(Y, lb = 0, vtype = GRB.CONTINUOUS, name = 'flow')

    # Set the objective function
    model.modelSense = GRB.MINIMIZE
    model.setObjective(quicksum(x[i,j]*cost[i][j] for i, j in A))

    # Constraint 1: leaving node i
    model.addConstrs(quicksum(x[i,j] for j in V if i != j) == 1 for i in V)
    # Constraint 2: entering node j
    model.addConstrs(quicksum(x[i,j] for i in V if i != j) == 1 for j in V)
    # Constraint 3: Flow Capacity Constraints
    model.addConstrs(quicksum(y[i,j,k] for j in V if i != j) - quicksum(y[j,i,k] for j in V if i != j) == 1 for i in V for k in V[1:] if i == 0)
    model.addConstrs(quicksum(y[i,j,k] for j in V if i != j) - quicksum(y[j,i,k] for j in V if i != j) == -1 for i in V for k in V[1:] if k == i)
    model.addConstrs(quicksum(y[i,j,k] for j in V if i != j) - quicksum(y[j,i,k] for j in V if i != j) == 0 for i in V for k in V[1:] if i != 0 and k != i)
    # Contraint 4: Ensuring flow is less than arc value
    model.addConstrs(y[i,j,k] <= x[i,j]*(len(V)-1) for i,j in A for k in V[1:])

    # Solve the model and print runtime
    model.Params.TimeLimit = 600
    model.optimize()
    print('The runtime was', model.Runtime, 'seconds.')

    ### UNCOMMENT LINE BELOW if you want to save a file of the formulation. Open with Microsoft Word.
    # model.write('Multi Flow.lp')
    
    ### UNCOMMENT BELOW if you want the positive decision variables printed out to the user 
    #for v in model.getVars():
     #   if v.x > 0:
          #  print(v)
            
    return model

In [7]:
def multiflow_LR(file_name):

    # Turn .dat file into a cost matrix with all costs associated from traversing from i to j
    verticesDF = TSP_vertices(file_name)
    cost = euclideanDistMatrix(verticesDF)

    # Create a list V that contains all the vertices, numbered 0 through n-1
    V = []
    for i in range(len(verticesDF)):
        V.append(i)
    
    # Create a list A of all tuples of (i,j) which represent the arc from i to j
    # Create a list Y that represents flow of commodity k through edge (i,j) 
    A = [(i, j) for i in V for j in V if i != j]
    Y = [(i, j ,k) for i in V for j in V for k in V[1:] if i != j]

    # Instantiate the Model
    model = Model('Multi Flow')
    model.ModelSense = GRB.MINIMIZE

    # Instantiate the decision variable. In this model, a linear relaxation is put on x, continuous between 0 and 1
    x = model.addVars(A, lb=0, ub=1, vtype = GRB.CONTINUOUS, name='edge')
    y = model.addVars(Y, lb = 0, vtype = GRB.CONTINUOUS, name = 'flow')

    # Set the objective function
    model.modelSense = GRB.MINIMIZE
    model.setObjective(quicksum(x[i,j]*cost[i][j] for i, j in A))

    # Constraint 1: leaving node i
    model.addConstrs(quicksum(x[i,j] for j in V if i != j) == 1 for i in V)
    # Constraint 2: entering node j
    model.addConstrs(quicksum(x[i,j] for i in V if i != j) == 1 for j in V)
    # Constraint 3: Flow Capacity Constraints
    model.addConstrs(quicksum(y[i,j,k] for j in V if i != j) - quicksum(y[j,i,k] for j in V if i != j) == 1 for i in V for k in V[1:] if i == 0)
    model.addConstrs(quicksum(y[i,j,k] for j in V if i != j) - quicksum(y[j,i,k] for j in V if i != j) == -1 for i in V for k in V[1:] if k == i)
    model.addConstrs(quicksum(y[i,j,k] for j in V if i != j) - quicksum(y[j,i,k] for j in V if i != j) == 0 for i in V for k in V[1:] if i != 0 and k != i)
    # Contraint 4: Ensuring flow is less than arc value
    model.addConstrs(y[i,j,k] <= x[i,j]*(len(V)-1) for i,j in A for k in V[1:])

    # Solve the model and print runtime
    model.Params.TimeLimit = 600
    model.optimize()
    print('The runtime was', model.Runtime, 'seconds.')

    ### UNCOMMENT LINE BELOW if you want to save a file of the formulation. Open with Microsoft Word.
    # model.write('Multi Flow.lp')
    
    ### UNCOMMENT BELOW if you want the positive decision variables printed out to the user 
    #for v in model.getVars():
     #   if v.x > 0:
          #  print(v)
            
    return model

The following are the Multicommodity Flow formulation integer and Linear Relaxation solutions for two instances of size 15.

In [8]:
multiflow('TSP_instance_n_15_s_1.dat')

Using license file /Users/student/gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3180 rows, 3150 columns and 12180 nonzeros
Model fingerprint: 0xc18c9a27
Variable types: 2940 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.06s
Presolved: 3180 rows, 3150 columns, 12180 nonzeros
Variable types: 2940 continuous, 210 integer (210 binary)
Found heuristic solution: objective 186.5019330
Found heuristic solution: objective 166.7177890

Root relaxation: objective 5.559851e+01, 1063 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Tim

<gurobi.Model MIP instance Multi Flow: 3180 constrs, 3150 vars, Parameter changes: TimeLimit=600.0>

In [9]:
multiflow_LR('TSP_instance_n_15_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3180 rows, 3150 columns and 12180 nonzeros
Model fingerprint: 0xf0659f6b
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 15 rows and 0 columns
Presolve time: 0.03s
Presolved: 3165 rows, 3150 columns, 11774 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1738894e+01   6.675000e+01   0.000000e+00      0s
    1130    5.5598512e+01   0.000000e+00   0.000000e+00      0s

Solved in 1130 iterations and 0.08 seconds
Optimal objective  5.559851187e+01
The runtime was 0.08617711067199707 seconds.


<gurobi.Model Continuous instance Multi Flow: 3180 constrs, 3150 vars, Parameter changes: TimeLimit=600.0>

In [11]:
multiflow('TSP_instance_n_15_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3180 rows, 3150 columns and 12180 nonzeros
Model fingerprint: 0x15bdc4ce
Variable types: 2940 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.05s
Presolved: 3180 rows, 3150 columns, 12180 nonzeros
Variable types: 2940 continuous, 210 integer (210 binary)
Found heuristic solution: objective 230.0914632
Found heuristic solution: objective 153.2243845

Root relaxation: objective 5.225179e+01, 1287 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   52.25179    0   16  153.22438   52.25179  65.9%     -    0s
H    0     0  

<gurobi.Model MIP instance Multi Flow: 3180 constrs, 3150 vars, Parameter changes: TimeLimit=600.0>

In [12]:
multiflow_LR('TSP_instance_n_15_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3180 rows, 3150 columns and 12180 nonzeros
Model fingerprint: 0x58ac9bab
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 15 rows and 0 columns
Presolve time: 0.03s
Presolved: 3165 rows, 3150 columns, 11774 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5739812e+01   4.275000e+01   0.000000e+00      0s
    1462    5.2251794e+01   0.000000e+00   0.000000e+00      0s

Solved in 1462 iterations and 0.10 seconds
Optimal objective  5.225179413e+01
The runtime was 0.10506510734558105 seconds.


<gurobi.Model Continuous instance Multi Flow: 3180 constrs, 3150 vars, Parameter changes: TimeLimit=600.0>

The following are the Multicommodity Flow formulation integer and Linear Relaxation solutions for two instances of size 20.

In [13]:
multiflow('TSP_instance_n_20_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 7640 rows, 7600 columns and 29640 nonzeros
Model fingerprint: 0xb49b0b67
Variable types: 7220 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.11s
Presolved: 7640 rows, 7600 columns, 29640 nonzeros
Variable types: 7220 continuous, 380 integer (380 binary)
Found heuristic solution: objective 239.3036164
Found heuristic solution: objective 202.1614181

Root relaxation: objective 4.693468e+01, 3077 iterations, 0.21 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   46.93468    0   25  202.16142   46.93468  76.8%     -    0s
     0     0  

<gurobi.Model MIP instance Multi Flow: 7640 constrs, 7600 vars, Parameter changes: TimeLimit=600.0>

In [34]:
multiflow_LR('TSP_instance_n_20_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 7640 rows, 7600 columns and 29640 nonzeros
Model fingerprint: 0x4164089f
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 20 rows and 0 columns
Presolve time: 0.08s
Presolved: 7620 rows, 7600 columns, 28899 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8979196e+01   6.462500e+01   0.000000e+00      0s
    4163    4.6934681e+01   0.000000e+00   0.000000e+00      1s

Solved in 4163 iterations and 0.65 seconds
Optimal objective  4.693468080e+01
The runtime was 0.6775219440460205 seconds.


<gurobi.Model Continuous instance Multi Flow: 7640 constrs, 7600 vars, Parameter changes: TimeLimit=600.0>

In [15]:
multiflow('TSP_instance_n_20_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 7640 rows, 7600 columns and 29640 nonzeros
Model fingerprint: 0x29eeb92a
Variable types: 7220 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.10s
Presolved: 7640 rows, 7600 columns, 29640 nonzeros
Variable types: 7220 continuous, 380 integer (380 binary)
Found heuristic solution: objective 310.5663684
Found heuristic solution: objective 185.0206061

Root relaxation: objective 6.126393e+01, 3334 iterations, 0.16 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   61.26393    0   31  185.02061   61.26393  66.9%     -    0s
     0     0  

<gurobi.Model MIP instance Multi Flow: 7640 constrs, 7600 vars, Parameter changes: TimeLimit=600.0>

In [16]:
multiflow_LR('TSP_instance_n_20_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 7640 rows, 7600 columns and 29640 nonzeros
Model fingerprint: 0x140db5af
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 20 rows and 0 columns
Presolve time: 0.04s
Presolved: 7620 rows, 7600 columns, 28899 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6713347e+01   6.462500e+01   0.000000e+00      0s
    3971    6.1263928e+01   0.000000e+00   0.000000e+00      0s

Solved in 3971 iterations and 0.27 seconds
Optimal objective  6.126392836e+01
The runtime was 0.27398204803466797 seconds.


<gurobi.Model Continuous instance Multi Flow: 7640 constrs, 7600 vars, Parameter changes: TimeLimit=600.0>

The following are the Multicommodity Flow formulation integer and Linear Relaxation solutions for two instances of size 25.

In [17]:
multiflow('TSP_instance_n_25_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15050 rows, 15000 columns and 58800 nonzeros
Model fingerprint: 0xc270f6d3
Variable types: 14400 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.17s
Presolved: 15050 rows, 15000 columns, 58800 nonzeros
Variable types: 14400 continuous, 600 integer (600 binary)
Found heuristic solution: objective 296.3893237

Root relaxation: objective 5.709035e+01, 6145 iterations, 0.50 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   57.09035    0   34  296.38932   57.09035  80.7%     -    0s
     0     0   60.63357    0   42  296.38932   60.63357 

<gurobi.Model MIP instance Multi Flow: 15050 constrs, 15000 vars, Parameter changes: TimeLimit=600.0>

In [18]:
multiflow_LR('TSP_instance_n_25_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15050 rows, 15000 columns and 58800 nonzeros
Model fingerprint: 0x07b32673
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 25 rows and 0 columns
Presolve time: 0.11s
Presolved: 15025 rows, 15000 columns, 57624 nonzeros

Ordering time: 0.11s

Barrier statistics:
 AA' NZ     : 2.287e+05
 Factor NZ  : 1.067e+06 (roughly 20 MBytes of memory)
 Factor Ops : 1.357e+08 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.22195396e+03  0.00000000e+00  1.79e+01 0.00e+00  4.16e-01     0s
   1 

<gurobi.Model Continuous instance Multi Flow_copy: 15050 constrs, 15000 vars, Parameter changes: TimeLimit=600.0>

In [19]:
multiflow('TSP_instance_n_25_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15050 rows, 15000 columns and 58800 nonzeros
Model fingerprint: 0xae869916
Variable types: 14400 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.19s
Presolved: 15050 rows, 15000 columns, 58800 nonzeros
Variable types: 14400 continuous, 600 integer (600 binary)
Found heuristic solution: objective 381.5008919

Root relaxation: objective 6.400423e+01, 7574 iterations, 0.87 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   64.00423    0   28  381.50089   64.00423  83.2%     -    1s
H    0     0                     217.7542795   64.00423 

<gurobi.Model MIP instance Multi Flow: 15050 constrs, 15000 vars, Parameter changes: TimeLimit=600.0>

In [20]:
multiflow_LR('TSP_instance_n_25_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15050 rows, 15000 columns and 58800 nonzeros
Model fingerprint: 0xc74a5a2d
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 25 rows and 0 columns
Presolve time: 0.14s
Presolved: 15025 rows, 15000 columns, 57624 nonzeros

Ordering time: 0.11s

Barrier statistics:
 AA' NZ     : 2.287e+05
 Factor NZ  : 1.067e+06 (roughly 20 MBytes of memory)
 Factor Ops : 1.357e+08 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.55568059e+03  0.00000000e+00  1.79e+01 0.00e+00  4.88e-01     0s
   1 

<gurobi.Model Continuous instance Multi Flow_copy: 15050 constrs, 15000 vars, Parameter changes: TimeLimit=600.0>

The following are the Multicommodity Flow formulation integer and Linear Relaxation solutions for two instances of size 30.

In [21]:
multiflow('TSP_instance_n_30_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 26160 rows, 26100 columns and 102660 nonzeros
Model fingerprint: 0x37826d6c
Variable types: 25230 continuous, 870 integer (870 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.30s
Presolved: 26160 rows, 26100 columns, 102660 nonzeros
Variable types: 25230 continuous, 870 integer (870 binary)
Found heuristic solution: objective 355.3966520

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 6.819483e+01, 13154 iterations, 3.66 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap

<gurobi.Model MIP instance Multi Flow: 26160 constrs, 26100 vars, Parameter changes: TimeLimit=600.0>

In [22]:
multiflow_LR('TSP_instance_n_30_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 26160 rows, 26100 columns and 102660 nonzeros
Model fingerprint: 0xd728a314
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 30 rows and 0 columns
Presolve time: 0.16s
Presolved: 26130 rows, 26100 columns, 100949 nonzeros

Ordering time: 0.33s

Barrier statistics:
 AA' NZ     : 4.642e+05
 Factor NZ  : 2.248e+06 (roughly 40 MBytes of memory)
 Factor Ops : 3.811e+08 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.55714925e+03  0.00000000e+00  1.92e+01 0.00e+00  3.23e-01     1s
   

<gurobi.Model Continuous instance Multi Flow_copy: 26160 constrs, 26100 vars, Parameter changes: TimeLimit=600.0>

In [23]:
multiflow('TSP_instance_n_30_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 26160 rows, 26100 columns and 102660 nonzeros
Model fingerprint: 0x335f815a
Variable types: 25230 continuous, 870 integer (870 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.36s
Presolved: 26160 rows, 26100 columns, 102660 nonzeros
Variable types: 25230 continuous, 870 integer (870 binary)
Found heuristic solution: objective 463.0080339

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 5.700967e+01, 13597 iterations, 3.27 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap

<gurobi.Model MIP instance Multi Flow: 26160 constrs, 26100 vars, Parameter changes: TimeLimit=600.0>

In [24]:
multiflow_LR('TSP_instance_n_30_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 26160 rows, 26100 columns and 102660 nonzeros
Model fingerprint: 0xee6c6e3d
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 30 rows and 0 columns
Presolve time: 0.13s
Presolved: 26130 rows, 26100 columns, 100949 nonzeros

Ordering time: 0.36s

Barrier statistics:
 AA' NZ     : 4.642e+05
 Factor NZ  : 2.248e+06 (roughly 40 MBytes of memory)
 Factor Ops : 3.811e+08 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.95488603e+03  0.00000000e+00  1.92e+01 0.00e+00  3.77e-01     1s
   

<gurobi.Model Continuous instance Multi Flow_copy: 26160 constrs, 26100 vars, Parameter changes: TimeLimit=600.0>

The following are the Multicommodity Flow formulation integer and Linear Relaxation solutions for two instances of size 50.

In [25]:
multiflow('TSP_instance_n_50_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 122600 rows, 122500 columns and 485100 nonzeros
Model fingerprint: 0xf4ba92da
Variable types: 120050 continuous, 2450 integer (2450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 1.44s
Presolved: 122600 rows, 122500 columns, 485100 nonzeros
Variable types: 120050 continuous, 2450 integer (2450 binary)
Found heuristic solution: objective 677.5179789

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   13206    1.0075928e+02   5.625502e-01   4.312828e+10      5s
   18276    9.9522980e+01   1.681822e-01   3.774977e+10     10s
   22571    9.9254945e+01   0.000000e+

     0     0  113.84010    0  119  114.18772  113.84010  0.30%     -  358s
     0     0  113.89021    0  126  114.18772  113.89021  0.26%     -  359s
     0     0  113.89021    0  124  114.18772  113.89021  0.26%     -  359s
     0     0  113.93993    0  116  114.18772  113.93993  0.22%     -  360s
     0     0  113.94006    0  118  114.18772  113.94006  0.22%     -  360s
     0     0  113.94895    0  110  114.18772  113.94895  0.21%     -  361s
     0     0  113.94912    0  111  114.18772  113.94912  0.21%     -  361s
     0     0  113.95473    0   98  114.18772  113.95473  0.20%     -  362s
     0     0  113.95473    0   98  114.18772  113.95473  0.20%     -  362s
     0     0  113.95473    0   96  114.18772  113.95473  0.20%     -  362s
     0     0  113.97572    0  110  114.18772  113.97572  0.19%     -  367s
     0     0  113.97572    0  101  114.18772  113.97572  0.19%     -  368s
     0     0  113.98538    0   89  114.18772  113.98538  0.18%     -  369s
     0     0  113.98538  

<gurobi.Model MIP instance Multi Flow: 122600 constrs, 122500 vars, Parameter changes: TimeLimit=600.0>

In [26]:
multiflow_LR('TSP_instance_n_50_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 122600 rows, 122500 columns and 485100 nonzeros
Model fingerprint: 0x675306e1
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 50 rows and 0 columns
Presolve time: 0.71s
Presolved: 122550 rows, 122500 columns, 480249 nonzeros

Elapsed ordering time = 5s
Ordering time: 9.46s

Barrier statistics:
 AA' NZ     : 3.414e+06
 Factor NZ  : 1.789e+07 (roughly 240 MBytes of memory)
 Factor Ops : 7.064e+09 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.07176147e+03  0.00000000e+00  4.20e+

<gurobi.Model Continuous instance Multi Flow_copy: 122600 constrs, 122500 vars, Parameter changes: TimeLimit=600.0>

In [27]:
multiflow('TSP_instance_n_50_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 122600 rows, 122500 columns and 485100 nonzeros
Model fingerprint: 0xea413c5f
Variable types: 120050 continuous, 2450 integer (2450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 1.68s
Presolved: 122600 rows, 122500 columns, 485100 nonzeros
Variable types: 120050 continuous, 2450 integer (2450 binary)
Found heuristic solution: objective 746.9465187

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   13422    1.3173801e+02   3.250869e+00   4.526866e+10      5s
   16542    1.3209986e+02   2.926968e+00   7.147602e+10     10s
   19662    1.3199111e+02   1.581558e+

<gurobi.Model MIP instance Multi Flow: 122600 constrs, 122500 vars, Parameter changes: TimeLimit=600.0>

In [28]:
multiflow_LR('TSP_instance_n_50_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 122600 rows, 122500 columns and 485100 nonzeros
Model fingerprint: 0x65c7c0c9
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 50 rows and 0 columns
Presolve time: 0.58s
Presolved: 122550 rows, 122500 columns, 480249 nonzeros

Elapsed ordering time = 5s
Ordering time: 9.06s

Barrier statistics:
 AA' NZ     : 3.414e+06
 Factor NZ  : 1.789e+07 (roughly 240 MBytes of memory)
 Factor Ops : 7.064e+09 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.31262876e+03  0.00000000e+00  4.20e+

<gurobi.Model Continuous instance Multi Flow_copy: 122600 constrs, 122500 vars, Parameter changes: TimeLimit=600.0>

The following are the Multicommodity Flow formulation integer and Linear Relaxation solutions for two instances of size 70.

In [29]:
multiflow('TSP_instance_n_70_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 338240 rows, 338100 columns and 1342740 nonzeros
Model fingerprint: 0x21518468
Variable types: 333270 continuous, 4830 integer (4830 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 3.95s
Presolved: 338240 rows, 338100 columns, 1342740 nonzeros
Variable types: 333270 continuous, 4830 integer (4830 binary)
Found heuristic solution: objective 991.0609016

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3677157e+02   1.700000e+01   9.353117e+09      7s
   21430    1.3683954e+02   5.846208e+00   8.358147e+10     10s
   31448    1.3692131e+02   4.689367

<gurobi.Model MIP instance Multi Flow: 338240 constrs, 338100 vars, Parameter changes: TimeLimit=600.0>

In [30]:
multiflow_LR('TSP_instance_n_70_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 338240 rows, 338100 columns and 1342740 nonzeros
Model fingerprint: 0x5147773d
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 1.40s
Elapsed ordering time = 5s
Elapsed ordering time = 10s
Elapsed ordering time = 15s
Elapsed ordering time = 20s
Elapsed ordering time = 25s
Elapsed ordering time = 30s
Elapsed ordering time = 35s
Elapsed ordering time = 40s
Elapsed ordering time = 45s
Ordering time: 48.22s

Barrier statistics:
 AA' NZ     : 1.284e+07
 Factor NZ  : 7.068e+07 (roughly 800 MBytes of memory)
 Factor Ops : 5.127e+10 (roughly 3 seconds per iteration)
 Threads    : 1

                  Objective               

<gurobi.Model Continuous instance Multi Flow_copy: 338240 constrs, 338100 vars, Parameter changes: TimeLimit=600.0>

In [31]:
multiflow('TSP_instance_n_70_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 338240 rows, 338100 columns and 1342740 nonzeros
Model fingerprint: 0x8a15a7a6
Variable types: 333270 continuous, 4830 integer (4830 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 4.01s
Presolved: 338240 rows, 338100 columns, 1342740 nonzeros
Variable types: 333270 continuous, 4830 integer (4830 binary)
Found heuristic solution: objective 1055.8697466

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4389762e+02   1.700000e+01   9.353117e+09      8s
   23728    1.4426386e+02   4.944263e+00   1.164621e+11     10s
   27732    1.4447266e+02   4.71509


Solution count 3: 157.368 164.91 1055.87 

Time limit reached
Best objective 1.573676296352e+02, best bound 1.087904441239e+02, gap 30.8686%
The runtime was 600.2076570987701 seconds.


<gurobi.Model MIP instance Multi Flow: 338240 constrs, 338100 vars, Parameter changes: TimeLimit=600.0>

In [32]:
multiflow_LR('TSP_instance_n_70_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 338240 rows, 338100 columns and 1342740 nonzeros
Model fingerprint: 0xacf998a8
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 1.33s
Elapsed ordering time = 5s
Elapsed ordering time = 10s
Elapsed ordering time = 15s
Elapsed ordering time = 20s
Elapsed ordering time = 25s
Elapsed ordering time = 30s
Elapsed ordering time = 35s
Elapsed ordering time = 40s
Elapsed ordering time = 45s
Elapsed ordering time = 50s
Ordering time: 54.80s

Barrier statistics:
 AA' NZ     : 1.284e+07
 Factor NZ  : 7.068e+07 (roughly 800 MBytes of memory)
 Factor Ops : 5.127e+10 (roughly 2 seconds per iteration)
 Threads    : 1

              

<gurobi.Model Continuous instance Multi Flow_copy: 338240 constrs, 338100 vars, Parameter changes: TimeLimit=600.0>